##### @Time    : 2019/4/6 
##### @Author  : ChanLiang
##### @Github  ：https://github.com/ChanLiang

# 决策树

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.datasets import load_iris

### 1. 创建数据集

In [2]:
################################################################################
''' 创建数据集，以 DataFrame 格式返回 '''

def create_data():
    
    ''' 创建数据集 和 每个维度的名称 '''
    
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    feature_names = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    
    ''' 用数据集 和 属性名 构造 DataFrame, 并返回 '''
    
    df = pd.DataFrame(datasets, columns = feature_names)
    
    return df

In [3]:
''' 创建训练数据集 '''

train_data = create_data()
train_data

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


### 2.先实现计算熵，条件熵的函数，进行一次基于信息增益的最基本的特征选择过程

In [37]:
#############################################################################################################
''' 计算某一列的熵 '''

def entropy(col):   # col:list类型
    
    ''' 记录该列中不同类别分别有多少个 '''
    
    counts = {}
    for c in col:
        counts[c] = counts.get(c, 0) + 1
    
    ''' 计算香农熵,并返回 '''
    
    return np.sum([-count/len(col) * math.log(count/len(col), 2) for count in counts.values()])
    

#############################################################################################################
''' 计算引入某一列特征后， Y这一列的 条件熵 大小'''

def conditional_entropy(dataset, col):  # dataset: ndarray    condition_col: int类型
    
    ''' 依据该属性值将原始数据集(Y这一列)划分成不同的子集 '''
    
    dic = {}
    for row in dataset:
        if row[col] not in dic:
            dic[row[col]] = []
        dic[row[col]].append(row[-1])
        
    ''' 计算不同子集的熵，乘上各自系数再加和，组成条件熵(用该属性划分后所得子系统的熵) '''
    
    conditional_entropy = np.sum([len(subset)/len(dataset) * entropy(subset) for subset in dic.values()])
    
    return conditional_entropy

#############################################################################################################
''' 选择最好的节点分裂属性 '''

def chooseBestFeatureToSplit(dataset):  # dataset: ndarray
    
    ''' 先计算整个系统(Y)的熵 '''
    
    ent = entropy(dataset[-1])
    
    ''' 依次计算所有属性分裂的 信息增益 ，并记录最佳属性(信息增益最大) '''
    
    max_info_gain, best_feature = 0, -1
    for i in range(len(dataset[0]) - 1):
        con_ent = conditional_entropy(dataset, i)
        info_gain = ent - con_ent
        if info_gain > max_info_gain:
            max_info_gain, best_feature = info_gain, i
            
    ''' 返回最佳分裂属性的下标 '''
    
    return best_feature

In [38]:
''' 计算下Y这一列的的 香农熵 大小 '''

ent = entropy(train_data['类别'].values)
ent

0.9709505944546686

In [41]:
''' 计算用各个属性划分后 信息增益 的大小 '''

for col in range(4):
    con_ent = conditional_entropy(train_data.values, col)
    print ('infomation gain of feature ' + train_data.columns[col] + ' is ', ent - con_ent)

infomation gain of feature 年龄 is  0.08300749985576883
infomation gain of feature 有工作 is  0.32365019815155627
infomation gain of feature 有自己的房子 is  0.4199730940219749
infomation gain of feature 信贷情况 is  0.36298956253708536


In [31]:
''' 计算数据集的首个 最佳分裂属性 '''

feature_index = chooseBestFeatureToSplit(train_data.values)
print ('最佳分裂属性为：', train_data.columns[feature_index])

最佳分裂属性为： 有自己的房子


### 3. 基本的函数都实现了，下一步把它们拼接成一个DT

##### 还缺哪些函数呢？一个是`节点分裂函数`，一个是`多数投票函数`，还有一个`递归建树的函数`

In [60]:
class DecisionTree(object):
    #############################################################################################################
    ''' 计算某一列的熵 '''

    def entropy(self, col):   # col:list类型

        ''' 记录该列中不同类别分别有多少个 '''

        counts = {}
        for c in col:
            counts[c] = counts.get(c, 0) + 1

        ''' 计算香农熵,并返回 '''

        return np.sum([-count/len(col) * math.log(count/len(col), 2) for count in counts.values()])


    #############################################################################################################
    ''' 计算引入某一列特征后， Y这一列的 条件熵 大小'''

    def conditional_entropy(self, dataset, col):  # dataset: ndarray    condition_col: int类型

        ''' 依据该属性值将原始数据集(Y这一列)划分成不同的子集 '''

        dic = {}
        for row in dataset:
            if row[col] not in dic:
                dic[row[col]] = []
            dic[row[col]].append(row[-1])

        ''' 计算不同子集的熵，乘上各自系数再加和，组成条件熵(用该属性划分后所得子系统的熵) '''

        conditional_entropy = np.sum([len(subset)/len(dataset) * self.entropy(subset) for subset in dic.values()])

        return conditional_entropy

    #############################################################################################################
    ''' 选择最好的节点分裂属性 '''

    def chooseBestFeatureToSplit(self, dataset):  # dataset: ndarray

        ''' 先计算整个系统(Y)的熵 '''

        ent = self.entropy(dataset[-1])

        ''' 依次计算所有属性分裂的 信息增益 ，并记录最佳属性(信息增益最大) '''

        max_info_gain, best_feature = 0, -1
        for i in range(len(dataset[0]) - 1):
            con_ent = self.conditional_entropy(dataset, i)
            info_gain = ent - con_ent
            if info_gain > max_info_gain:
                max_info_gain, best_feature = info_gain, i

        ''' 返回最佳分裂属性的下标 '''

        return best_feature
    
    #############################################################################################################
    ''' 节点分裂函数(依据某一列属性) '''
    
    def splitNode(self, dataset, col):   # dataset: ndarray, col: int
        
        ''' 先看看这一列属性有几个取值 '''
        
        feature_vals = set([row[col] for row in dataset])
        
        ''' 有几个值就能划分出几个子节点, 组成子节点列表返回 '''
        
        subset_list, val_list = [], []
        for val in feature_vals:
            subset = []
            for row in dataset:
                if row[col] == val:
                    row = list(row) # 坑：这里不把ndarray转化成一般list的话，后续切片拼接会出现很奇怪的答案
                    row = row[:col] + row[col + 1:]
                    subset.append(row)
            subset_list.append(subset)
            val_list.append(val)
            
        return subset_list, val_list
    
    #############################################################################################################
    ''' 多数类投票函数: 用于决定不纯子节点的类别 '''
    
    def majorityVoting(self, dataset):   # dataset: ndarray
        
        ''' 统计该节点中不同类别出现的次数 '''
        
        class_counts = {}
        for row in dataset:
            class_counts[row[-1]] = class_counts.get(row[-1], 0) + 1
        
        ''' 对字典排序 并返回出现最多的类 '''
        
        tuple_list = sorted(class_counts.items(), key = lambda e : e[1], reverse = True)
        
        return tuple_list[0][0]
    
    #############################################################################################################
    ''' 递归构建一棵二叉树 '''
    
    def createTree(self, dataset, feature_names):   # dataset: ndarray,   feature_names: list
        
        ''' 递归终止条件 '''
        
        class_list = [row[-1] for row in dataset]
        if len(dataset[0]) == 1:
            return self.majorityVoting(dataset)
        if len(set(class_list)) == 1:
            return class_list[0]
        
        ''' 找到最佳分裂属性 '''
        
        best_feature_col = self.chooseBestFeatureToSplit(dataset)
        best_feature = feature_names[best_feature_col]
        
        ''' 用该属性分裂节点 '''
        
        subset_list, val_list = self.splitNode(dataset, best_feature_col)
        tree = {best_feature:{}}
        for i in range(len(val_list)):
            tree[best_feature][val_list[i]] = self.createTree(subset_list[i], feature_names[:best_feature_col] + feature_names[best_feature_col + 1:])
        
        return tree
    


In [61]:
''' 测试 划分子节点函数 '''

model = DecisionTree()
model.splitNode(train_data.values, 2)

([[['青年', '否', '一般', '否'],
   ['青年', '否', '好', '否'],
   ['青年', '是', '好', '是'],
   ['青年', '否', '一般', '否'],
   ['中年', '否', '一般', '否'],
   ['中年', '否', '好', '否'],
   ['老年', '是', '好', '是'],
   ['老年', '是', '非常好', '是'],
   ['老年', '否', '一般', '否']],
  [['青年', '是', '一般', '是'],
   ['中年', '是', '好', '是'],
   ['中年', '否', '非常好', '是'],
   ['中年', '否', '非常好', '是'],
   ['老年', '否', '非常好', '是'],
   ['老年', '否', '好', '是']]],
 ['否', '是'])

In [62]:
''' 测试投票函数 '''

model.majorityVoting(train_data.values)

'是'

In [63]:
''' 测试建树函数 '''

model.createTree(train_data.values, train_data.columns)

{'有自己的房子': {'否': {'有工作类别': {'否': '否', '是': '是'}}, '是': '是'}}